In [1]:
import sys
if "../.." not in sys.path: sys.path.insert(0, "../..")
if ".." not in sys.path: sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from experiment.v1dd_client import V1DDClient
from experiment.v1dd_ophys_session import V1DDOPhysSession
from stimulus_analysis.locally_sparse_noise import LocallySparseNoise
from stimulus_analysis.timing_utils import find_nearest, get_frame_window_from_time_window
from statsmodels.sandbox.stats.multicomp import multipletests

import scipy.interpolate as si
import scipy.ndimage.filters as filt
import scipy.stats as stats

from analysis_tools import set_stylesheet
set_stylesheet()

%load_ext autoreload
%autoreload 2

/var/folders/8l/944wpj6s7sq8gcv57wcd3q6h0000gn/T/ipykernel_16106/4270177090.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# windows
# base_folder = r"\\allen\programs\mindscope\workgroups\surround\v1dd_in_vivo_new_segmentation\data"

# # linux and mac
# base_folder = "/allen/programs/mindscope/workgroups/surround/v1dd_in_vivo_new_segmentation/data"
# base_folder = "/Volumes/programs/mindscope/workgroups/surround/v1dd_in_vivo_new_segmentation/data"
base_folder = "/Users/chase/Desktop/test_v1dd_data"
client = V1DDClient(base_folder)

In [3]:
mouse = 409828
col, vol = 1, 3
sess = client.load_ophys_session(mouse=mouse, column=col, volume=vol)
print(f"Loaded ophys session {sess.get_session_id()} (mouse {sess.get_mouse_id()}, column {sess.get_column_id()}, volume {sess.get_volume_id()})")

Loaded ophys session M409828_13 (mouse 409828, column 1, volume 3)


In [4]:
plane = sess.get_planes()[0]
print(f"Analyzing plane {plane}")

Analyzing plane 1


In [5]:
lsn = LocallySparseNoise(sess, plane)

In [6]:
def get_disc_masks(lsn, radius=3):
    '''
    Obtain an indicator mask surrounding each pixel. The mask is a square, excluding pixels which 
    are coactive on any trial with the main pixel.
    Parameters
    ----------
    LSN_template : np.ndarray
        Dimensions are (nTrials, nYPixels, nXPixels). Luminance values per pixel 
        and trial.
    radius : int
        The base mask will be a box whose sides are 2 * radius + 1 in length.
    on_luminance : int, optional
        The value of the luminance for on trials. Default is 255
    off_luminance : int, optional
        The value of the luminance for off trials. Default is 0
  
    Returns
    -------
    masks : np.ndarray
        Dimensions are (nYPixels, nXPixels, nYPixels, nXPixels). The first 2 
        dimensions describe the pixel from which the mask was computed. The last 
        2 serve as the dimensions of the mask images themselves. Masks are binary 
        arrays of type float, with 1 indicating inside, 0 outside.
    '''

    ydim, xdim = lsn.image_shape
    
    # convert template to true on trials a pixel is not gray and false when gray
    nongray_pixels = np.where(lsn.trial_template != lsn.pixel_gray, 1, 0) # shape is (n_trials, ydim, xdim)

    # get number of trials each pixel is not gray
    num_nongray_trials = nongray_pixels.sum(axis=0).astype(float) # shape is (ydim, xdim)

    masks = np.zeros((ydim, xdim, ydim, xdim), dtype=float)

    for y in range(ydim):
        for x in range(xdim):
            trials_nongray = np.argwhere(nongray_pixels[:, y, x] > 0)[:, 0] # indices on axis=0 where there are nongray pixels at (y, x)
            num_times_nongray = nongray_pixels[trials_nongray, :, :].sum(axis=0) # shape (ydim, xdim); number of times each pixel is nongray during these trials
            raw_mask = num_times_nongray / num_nongray_trials

            # For this stimulus (center_y, center_x) = (y, x) but including for completeness
            center_y, center_x = np.unravel_index(raw_mask.argmax(), (ydim, xdim))

            x_max = min(center_x + radius + 1, xdim)
            y_max = min(center_y + radius + 1, ydim)
            x_min = max(center_x - radius, 0)
            y_min = max(center_y - radius, 0)

            masks[y, x, y_min:y_max, x_min:x_max] = raw_mask[y_min:y_max, x_min:x_max] == 0 # include nearby pixels in mask if they are always gray
            masks[y, x, center_y, center_x] = 1 # include center in mask

    return masks

In [18]:
def build_trial_matrix(lsn):
    n_trials, ydim, xdim = lsn.trial_template.shape
    on_off_luminance = (lsn.pixel_on, lsn.pixel_off)
    trial_matrix = np.zeros((n_trials, ydim, xdim, 2), dtype=bool)

    for y in range(ydim):
        for x in range(xdim):
            for i, on_off in enumerate(on_off_luminance):
                trials_with_pixel_on_off = np.argwhere(lsn.trial_template[:, y, x] == on_off)[:, 0] # trial indices where the (y, x) pixel value is on_off
                trial_matrix[trials_with_pixel_on_off, y, x, i] = True
    
    # Compute the number of trials in which each pixel is on/off
    trials_per_pixel = np.sum(trial_matrix, axis=0) # shape (ydim, xdim, 2)

    # reshape to (n_trials, ydim*xdim*2)
    trial_matrix = trial_matrix.reshape(n_trials, ydim*xdim*2)
    
    return trial_matrix, trials_per_pixel

In [63]:
active_trials.reshape(1, len(active_trials)).shape[0]

1

In [68]:
np.unique(active_pixels)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [83]:
active_trials.reshape(1, -1)[:, active_pixels == 0]

array([[  25,   73,   89,  149,  189,  229,  258,  311,  337,  372,  425,
         463,  494,  530,  549,  584,  633,  695,  732,  770,  809,  846,
         882,  903,  954, 1001, 1018, 1054, 1122, 1130, 1204, 1237, 1274,
        1318, 1349, 1388, 1431, 1475, 1508, 1521, 1587, 1618, 1666, 1682]])

In [84]:
lsn.sweep_responses

array([[0.        , 0.        , 0.04349729, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [86]:
lsn.sweep_responses.shape

(1705, 1234)

In [87]:
lsn.sweep_responses

array([[0.        , 0.        , 0.04349729, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [100]:
a = np.arange(5)
print(a)
np.where(a == 1)[0]

[0 1 2 3 4]


array([1])

In [119]:
from stimulus_analysis.proba_utils import *

# Step 1: Compute categories
# Two categories are defined for each pixel -- one for pixel on and one for pixel off
n_categories = lsn.n_pixels * 2
sweep_categories = np.arange(n_categories) # shape (n_categories,)
sweep_category_matrix = lsn.design_matrix.T # shape (n_sweeps, n_categories)

# Step 2: Compute observed and expected responses, along with actual chi^2 statistic
sweep_responses = lsn.sweep_responses
observed = compute_chisq_observed(sweep_responses, sweep_category_matrix) # shape (n_rois, n_categories)
expected = compute_chisq_expected(sweep_responses) # shape (n_rois, 1)
chisq_actual = compute_chisq_statistic(observed, expected)

In [127]:
# Step 3: Compute shuffled distribution
n_rois = lsn.n_rois
n_sweeps = lsn.n_sweeps
n_shuffles = 1000

chisq_shuffle = np.zeros((n_rois, n_shuffles))

for shuffle_i in tqdm(range(n_shuffles)):
    shuffle_stim_idx = np.random.choice(n_sweeps, size=n_sweeps, replace=True) # shape (n_sweeps,)
    shuffle_sweep_events = sweep_responses[shuffle_stim_idx] # shape (n_sweeps, n_rois)

    shuffle_observed = compute_chisq_observed(shuffle_sweep_events, sweep_category_matrix)
    shuffle_expected = compute_chisq_expected(shuffle_sweep_events)
    chisq_shuffle[:, shuffle_i] = compute_chisq_statistic(shuffle_observed, shuffle_expected)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [135]:
masks = get_disc_masks(lsn, radius=2)
masks = masks.reshape(lsn.n_pixels, lsn.n_pixels)
masks.shape

mask_center_pixel, pixels_in_mask = np.where(masks)

chi_sum = np.zeros((n_rois, lsn.n_pixels))

(array([  0,   0,   0, ..., 111, 111, 111]),
 array([  0,   1,   2, ..., 109, 110, 111]))

In [128]:
# Step 4: Compute p-values
# For ROI i, the p-value is computed as the fraction of times the shuffled chi^2 value is greater than the actual
p_values = np.mean(chisq_shuffle > np.expand_dims(chisq_actual, axis=1), axis=1)

(p_values <= 0.05).mean()

0.0713128038897893

In [110]:
def calc_observed(lsn, active_trials, active_pixels, n_pixels):
    # If necessary, convert active_trials into 2D matrix (where rows are indices)
    if active_trials.ndim == 1:
        active_trials = active_trials.reshape(-1, 1)
    n_trial_repeats, _ = active_trials.shape

    observed_by_pixel = np.zeros((lsn.n_rois, n_pixels, n_trial_repeats))
    unique_active_pixels = np.unique(active_pixels)

    for pixel in unique_active_pixels:
        trials_pixel_is_displayed = active_trials[:, active_pixels == pixel] # shape (n_trial_repeats,)
        response_mat = lsn.sweep_responses[trials_pixel_is_displayed, :] # shape (n_trial_repeats, n_rois)
        response_mat = response_mat.T # shape (n_rois, n_trial_repeats); move n_rois to dim 0
        observed_by_pixel[:, pixel, :] = response_mat
        
#         #show progress during shuffling
#         if num_reps > 1:
#             print 'pixel ' + str(p) + ' of ' + str(len(pixels_to_populate))
    
    return 

def chi_square_across_template(lsn, expected_by_pixel, trial_matrix, num_shuffles):
    # Array shapes
    n_trials, n_pixels = trial_matrix.shape
    active_trials, active_pixels = np.where(trial_matrix) # all pairings p[i] = (active_trials[i], active_pixels[i]) such that trial_matrix[p[i]] = True
    
    # calculate pixelwise contributions to chi_sum for the actual data
    observed_by_pixel_actual = calc_observed(responses,trials.reshape(1,len(trials)),pixels,num_pixels)
    chi_actual = compute_chi(observed_by_pixel_actual,expected_by_pixel)
    
    #shuffle the trial labels to build null distribution
    trial_labels = np.random.choice(num_trials,size=(num_shuffles,num_trials))
    shuffled_trials = np.zeros((num_shuffles,len(trials))).astype(int)
    for s in range(num_shuffles):
        shuffled_trials[s,:] = trial_labels[s,trials]

    # calculate pixelwise contributions to chi_sum for the shuffled data
    observed_by_pixel_shuffled = calc_observed(responses,shuffled_trials,pixels,num_pixels)
    chi_shuffle = compute_chi(observed_by_pixel_shuffled,expected_by_pixel)
    
    return chi_actual, chi_shuffle

In [60]:
np.where(trial_matrix)

(array([   0,    0,    0, ..., 1704, 1704, 1704]),
 array([ 45, 107, 116, ..., 116, 128, 179]))

In [56]:
def chi_square_receptive_fields(lsn, num_shuffles=1000):
    # Array shapes
    n_trials, n_rois = lsn.sweep_responses.shape
    n_trials, ydim, xdim = lsn.trial_template.shape

    # define the masks for exclusion regions we'll calculate the chi sq sum over
    masks = get_disc_masks(lsn, radius=2)
    
    # build a binary matrix of pixels displayed on each trial
    trial_matrix, trials_per_pixel = build_trial_matrix(lsn)
    # trial_matrix is shape (n_trials, ydim*xdim*2)
    # trials_per_pixel is shape (ydim, xdim, 2)
    
    # Pre-calculate the expected values (since it doesn't change with shuffling)
    # Each expected value is computed from the ROI's mean response across all trials,
    # weighted by the number of times each pixel is shown as nongray.
    # Importantly, this metric ignores when each pixel is shown, so it can be used to
    # identify outlier responses that hint at receptive field
    roi_mean_trial_response = np.mean(lsn.sweep_responses, axis=0) # shape (n_rois,)
    expected_by_pixel = roi_mean_trial_response.reshape(n_rois, 1, 1) * trials_per_pixel.reshape(1, ydim*xdim*2, 1) # shape (n_rois, ydim*xdim*2, 1)
    # expected_by_pixel[roi, pixel, 0] = roi mean response * number of trials pixel is on/off
    
    # calculate the contribution of each pixel to the chi_sum test statistic,
    # for the actual data as well as shuffles
    chi_actual, chi_shuffle = chi_square_across_template(responses,expected_by_pixel,trial_matrix,num_shuffles)
    
    #apply region masks around each pixel to sum pixel-wise chi contributions
    # to the chi_sum test statistic, then calculate p_values
    p_values = chi_square_across_masks(masks,chi_actual,chi_shuffle,num_y,num_x)
    
    # p_values should be no smaller than the 1.0 over number of shuffles
    p_values = np.where(p_values==0.0,1.0/num_shuffles,p_values)
    
    return p_values